In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_10%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,NaN,0.500,6.68,NaN,0.2,...,0.4,NaN,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1.0
1,3.89,0.00,NaN,0.000,4.83,NaN,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0.0,NaN,0.0,NaN,1.416,NaN,NaN,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,NaN,0.2,NaN,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,NaN,2.0,1.578,0.4,0.5,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,NaN,0.074,NaN,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,NaN,0.0
773,55.57,0.24,0.2,NaN,NaN,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,NaN,0.040,0.00,0.2,0.000,0.20,5.0,NaN,...,0.4,NaN,0.2,0.0,2.0,NaN,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      74
PARA_A            62
Score_A           76
Risk_A            87
PARA_B            77
Score_B           83
Risk_B            93
TOTAL             94
numbers           88
Score_B.1         94
Risk_C            91
Money_Value       71
Score_MV          84
Risk_D            71
District_Loss     80
PROB              63
RiSk_E            93
History           73
Prob              89
Risk_F            80
Score             75
Inherent_Risk     71
CONTROL_RISK      92
Detection_Risk    75
Audit_Risk        81
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 2017
persentase : 9.997026169706581


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score       9.536082
PARA_A             7.989691
Score_A            9.793814
Risk_A            11.211340
PARA_B             9.922680
Score_B           10.695876
Risk_B            11.984536
TOTAL             12.113402
numbers           11.340206
Score_B.1         12.113402
Risk_C            11.726804
Money_Value        9.149485
Score_MV          10.824742
Risk_D             9.149485
District_Loss     10.309278
PROB               8.118557
RiSk_E            11.984536
History            9.407216
Prob              11.469072
Risk_F            10.309278
Score              9.664948
Inherent_Risk      9.149485
CONTROL_RISK      11.855670
Detection_Risk     9.664948
Audit_Risk        10.438144
Risk               0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,9)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.89,4.18,0.6,2.508,2.50,0.0,0.500,6.68,0.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148
1,3.89,0.00,0.0,0.000,4.83,0.0,0.966,4.83,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.0,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.2,0.4,0.0,0.0,0.0,0.0,1.416,0.0,0.0,0.2832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.00,0.2,0.000,0.40,0.2,0.080,0.89,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.5,0.3156
772,55.57,0.47,0.2,0.094,0.37,0.0,0.074,0.00,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.0000
773,55.57,0.24,0.2,0.000,0.00,0.2,0.008,0.28,5.0,0.2,...,0.2,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912
774,55.57,0.20,0.0,0.040,0.00,0.2,0.000,0.20,5.0,0.0,...,0.2,0.4,0.0,0.2,0.0,2.0,0.000,0.4,0.5,0.2880


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4780


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,9)

Kolom : 1
[2.72, 3.89, 3.89, 3.89, 3.41, 3.89, 3.89, 3.41, 3.89]
mean :  3.6533333333333338
standar deviasi :  0.3837823457233135
fuzzy :  [(2.72, 0.05198267086799132), (3.89, 0.8268594092969731), (3.89, 0.8268594092969731), (3.89, 0.8268594092969731), (3.41, 0.8179267628392082), (3.89, 0.8268594092969731), (3.89, 0.8268594092969731), (3.41, 0.8179267628392082), (3.89, 0.8268594092969731)]
fuzzy :  None
pembilang :  25.018552000315687
penyebut :  6.648992652328245
rata-rata tertimbang :  3.7627582565541053
 
[55.57, 55.57, 59.85, 3.89, 2.72, 55.57, 1.99, 59.85, 55.57]
mean :  38.95333333333333
standar deviasi :  25.57423964330774
fuzzy :  [(55.57, 0.8097249334475057), (55.57, 0.8097249334475057), (59.85, 0.716204048003642), (3.89, 0.3907140246300099), (2.72, 0.36657826208261657), (55.57, 0.8097249334475057), (1.99, 0.35190720863173874), (59.85, 0.716204048003642), (55.57, 0.8097249334475057)]
fuzzy :  None
pembilang :  268.93254852660016
penyebut :  5.780507325141673
rata-rata tertimba

[0.0, 0.0, 2.72, 2.72, 3.89, 3.89, 3.89, 3.89, 2.72]
mean :  2.6355555555555554
standar deviasi :  1.4984518760011474
fuzzy :  [(0.0, 0.2129665087651319), (0.0, 0.2129665087651319), (2.72, 0.9984135119355012), (2.72, 0.9984135119355012), (3.89, 0.7044186941837126), (3.89, 0.7044186941837126), (3.89, 0.7044186941837126), (3.89, 0.7044186941837126), (2.72, 0.9984135119355012)]
fuzzy :  None
pembilang :  19.10780913889226
penyebut :  6.238848330071618
rata-rata tertimbang :  3.062714162610989
 
[3.89, 2.72, 2.37, 0.0, 3.89, 2.37, 3.89, 3.89, 3.89]
mean :  2.99
standar deviasi :  1.2397042658096595
fuzzy :  [(3.89, 0.7683608015307715), (2.72, 0.9765643544870511), (2.37, 0.8824557099608102), (0.0, 0.05457135739042138), (3.89, 0.7683608015307715), (2.37, 0.8824557099608102), (3.89, 0.7683608015307715), (3.89, 0.7683608015307715), (3.89, 0.7683608015307715)]
fuzzy :  None
pembilang :  21.783712699192527
penyebut :  6.637851139452952
rata-rata tertimbang :  3.2817416723490727
 
[2.72, 0.0, 3.8

[0.0, 2.72, 55.57, 55.57, 23.12947672728555, 3.89, 55.57, 55.57, 2.72]
mean :  28.304386303031723
standar deviasi :  25.18102600891921
fuzzy :  [(0.0, 0.5317067003590829), (2.72, 0.5968477636630625), (55.57, 0.5564656225143145), (55.57, 0.5564656225143145), (23.12947672728555, 0.9791067143356496), (3.89, 0.6250212775245217), (55.57, 0.5564656225143145), (55.57, 0.5564656225143145), (2.72, 0.5968477636630625)]
fuzzy :  None
pembilang :  152.0155891391347
penyebut :  5.555392709602637
rata-rata tertimbang :  27.363608134555797
 
[0.0, 21.61, 1.99, 3.41, 3.41, 55.57, 55.57, 2.37, 3.89]
mean :  16.424444444444443
standar deviasi :  21.76425668695815
fuzzy :  [(0.0, 0.7522245046238633), (21.61, 0.9720179457728293), (1.99, 0.8025940913468409), (3.41, 0.8363003184033516), (3.41, 0.8363003184033516), (55.57, 0.19842515406351413), (55.57, 0.19842515406351413), (2.37, 0.8118171688762846), (3.89, 0.8471952994795124)]
fuzzy :  None
pembilang :  55.57860624927298
penyebut :  6.255299955033062
rata-

[0.0, 1.85, 1.85, 1.85, 1.85, 1.85, 3.89, 1.85, 1.85]
mean :  1.8711111111111112
standar deviasi :  0.9177320408296924
fuzzy :  [(0.0, 0.1251522955655407), (1.85, 0.9997354803615401), (1.85, 0.9997354803615401), (1.85, 0.9997354803615401), (1.85, 0.9997354803615401), (1.85, 0.9997354803615401), (3.89, 0.08896985988251133), (1.85, 0.9997354803615401), (1.85, 0.9997354803615401)]
fuzzy :  None
pembilang :  13.292667225624916
penyebut :  7.212270517978833
rata-rata tertimbang :  1.843062762619455
 
[0.0, 1.99, 1.85, 2.37, 1.99, 2.37, 1.85, 3.89, 1.85]
mean :  2.017777777777778
standar deviasi :  0.9375355137306387
fuzzy :  [(0.0, 0.0986897617781278), (1.99, 0.9995612175813989), (1.85, 0.9841164970648614), (2.37, 0.9318681236032424), (1.99, 0.9995612175813989), (2.37, 0.9318681236032424), (1.85, 0.9841164970648614), (3.89, 0.1361877736789365), (1.85, 0.9841164970648614)]
fuzzy :  None
pembilang :  14.386925550174382
penyebut :  7.050085709020931
rata-rata tertimbang :  2.040673850498811
 


[0.0, 55.57, 2.37, 1.99, 3.89, 3.89, 1.99, 3.89, 2.34]
mean :  8.436666666666666
standar deviasi :  16.706079399095675
fuzzy :  [(0.0, 0.8802914810878584), (55.57, 0.018694352277309915), (2.37, 0.9361973790928177), (1.99, 0.9282568106044139), (3.89, 0.9636465345370824), (3.89, 0.9636465345370824), (1.99, 0.9282568106044139), (3.89, 0.9636465345370824), (2.34, 0.9355856273116556)]
fuzzy :  None
pembilang :  20.38712047666268
penyebut :  7.518222064589717
rata-rata tertimbang :  2.711694374217083
 
[0.0, 3.41, 0.0, 3.89, 55.57, 55.57, 55.57, 55.57, 55.57]
mean :  31.68333333333333
standar deviasi :  26.73404446269463
fuzzy :  [(0.0, 0.4954967823091095), (3.41, 0.5716790918903674), (0.0, 0.4954967823091095), (3.89, 0.5825430528354829), (55.57, 0.6709081848196049), (55.57, 0.6709081848196049), (55.57, 0.6709081848196049), (55.57, 0.6709081848196049), (55.57, 0.6709081848196049)]
fuzzy :  None
pembilang :  190.6273573310034
penyebut :  5.499756633442093
rata-rata tertimbang :  34.6610532131

...

[0.4004, 0.3892, 0.4156, 0.4292, 0.408, 0.3932, 0.3884, 0.4456, 0.4008]
mean :  0.40782222222222214
standar deviasi :  0.01821756037928349
fuzzy :  [(0.4004, 0.9203626935889884), (0.3892, 0.5930922856337284), (0.4156, 0.9129001319439974), (0.4292, 0.5023563050932526), (0.408, 0.999952390990584), (0.3932, 0.7246361650433503), (0.3884, 0.5665138023499569), (0.4456, 0.11649632995003882), (0.4008, 0.928408243615062)]
fuzzy :  None
pembilang :  2.531315626182208
penyebut :  6.264718348208959
rata-rata tertimbang :  0.4040589672967332
 
[2.8984, 6.4692, 6.036, 5.7568, 8.58, 1.8304, 2.7164, 3.5752, 4.3128]
mean :  4.686133333333334
standar deviasi :  2.052494278784826
fuzzy :  [(2.8984, 0.6843500248028411), (6.4692, 0.6857047186321232), (6.036, 0.805539230994293), (5.7568, 0.872806898573278), (8.58, 0.16540118204464682), (1.8304, 0.3799088410405722), (2.7164, 0.6310040112983584), (3.5752, 0.8637550305514925), (4.3128, 0.9835953195326953)]
fuzzy :  None
pembilang :  27.465024089834582
penyebut

[0.0, 0.294, 0.2904, 0.0, 0.28, 0.0, 0.298, 0.2884, 0.28]
mean :  0.19231111111111113
standar deviasi :  0.13609418670288967
fuzzy :  [(0.0, 0.36851086869967586), (0.294, 0.7564482143155279), (0.2904, 0.7712767078008624), (0.0, 0.36851086869967586), (0.28, 0.8125678667405248), (0.0, 0.36851086869967586), (0.298, 0.7396990083818636), (0.2884, 0.7794043364211601), (0.28, 0.8125678667405248)]
fuzzy :  None
pembilang :  1.3466230514504876
penyebut :  5.777496606499492
rata-rata tertimbang :  0.23308071699005092
 
[0.0, 11.0284, 149.8176, 45.2368, 12.7968, 14.7472, 34.9296, 7.36, 21.2664]
mean :  33.02031111111111
standar deviasi :  43.343566433899696
fuzzy :  [(0.0, 0.7481436227669319), (11.0284, 0.8792316831096089), (149.8176, 0.026508520153568214), (45.2368, 0.9610620791213496), (12.7968, 0.8968738455133226), (14.7472, 0.9149747155367999), (34.9296, 0.9990303648798391), (7.36, 0.8392669952596572), (21.2664, 0.9639021428377271)]
fuzzy :  None
pembilang :  143.68522962365324
penyebut :  7.

[0.0, 0.3756, 0.3476, 0.3432, 0.3376, 0.33, 0.0, 0.326, 0.308]
mean :  0.2631111111111111
standar deviasi :  0.14167804224480585
fuzzy :  [(0.0, 0.1783081326127092), (0.3756, 0.7296678596802353), (0.3476, 0.8371141744682525), (0.3432, 0.8523494681599232), (0.3376, 0.8709259228521419), (0.33, 0.8945481064530151), (0.0, 0.1783081326127092), (0.326, 0.9061892199732947), (0.308, 0.9510508629020522)]
fuzzy :  None
pembilang :  2.0351372907830507
penyebut :  6.398461879714334
rata-rata tertimbang :  0.3180666430529569
 
[0.0, 0.33, 0.3432, 0.3376, 0.326, 0.3476, 0.308, 0.3072, 0.3044]
mean :  0.28933333333333333
standar deviasi :  0.10340479249585635
fuzzy :  [(0.0, 0.019956945274265038), (0.33, 0.9255888464320411), (0.3432, 0.8731307137135292), (0.3376, 0.8967952718387349), (0.326, 0.9390732873346047), (0.3476, 0.8532189125809247), (0.308, 0.9838398941977348), (0.3072, 0.9851852715964378), (0.3044, 0.9894421559847135)]
fuzzy :  None
pembilang :  2.417435444855099
penyebut :  7.4662312989529

[0.0, 0.0, 0.312, 0.0, 0.3104, 0.3168, 0.3188, 0.3136, 0.3092]
mean :  0.2089777777777778
standar deviasi :  0.1477956680900115
fuzzy :  [(0.0, 0.36804735166453995), (0.0, 0.36804735166453995), (0.312, 0.7843342066555518), (0.0, 0.36804735166453995), (0.3104, 0.7902284053149539), (0.3168, 0.7663750504411746), (0.3188, 0.7587777567636889), (0.3136, 0.778392750772008), (0.3092, 0.794616993882112)]
fuzzy :  None
pembilang :  1.4644846754727727
penyebut :  5.77686721882311
rata-rata tertimbang :  0.2535084536305344
 
[0.0, 0.28, 0.28, 0.28, 0.28, 0.28, 0.28, 0.28, 0.2808]
mean :  0.2489777777777778
standar deviasi :  0.08802729094216907
fuzzy :  [(0.0, 0.01832382512324588), (0.28, 0.9397963597911225), (0.28, 0.9397963597911225), (0.28, 0.9397963597911225), (0.28, 0.9397963597911225), (0.28, 0.9397963597911225), (0.28, 0.9397963597911225), (0.28, 0.9397963597911225), (0.2808, 0.9367528356173789)]
fuzzy :  None
pembilang :  2.105041061431961
penyebut :  7.533651179278483
rata-rata tertimbang

[0.0, 0.30192199917010887, 0.3148, 0.3152, 0.3284, 0.3216, 0.3156, 0.3136, 0.3124]
mean :  0.2803913332411232
standar deviasi :  0.09935820481261155
fuzzy :  [(0.0, 0.01865764018715635), (0.30192199917010887, 0.9767969779879117), (0.3148, 0.9418033015012809), (0.3152, 0.940483683810871), (0.3284, 0.8898315587317229), (0.3216, 0.9175949259362121), (0.3156, 0.9391506954101346), (0.3136, 0.9456813175424932), (0.3124, 0.9494368153032782)]
fuzzy :  None
pembilang :  2.364721526649313
penyebut :  7.5194369164110615
rata-rata tertimbang :  0.31448119758652976
 
[0.0, 0.3136, 0.3012, 0.3184, 0.3156, 0.3152, 0.3148, 0.2696521244583077, 0.3156]
mean :  0.2737835693842564
standar deviasi :  0.09786444277776568
fuzzy :  [(0.0, 0.019984064673904896), (0.3136, 0.9205756919558109), (0.3012, 0.9615227014984944), (0.3184, 0.9013048115363128), (0.3156, 0.9127633670217257), (0.3152, 0.914351056526179), (0.3148, 0.915926207786319), (0.2696521244583077, 0.9991093940580407), (0.3156, 0.9127633670217257)]
fu

In [15]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
0,3.890000,4.180000,0.600000,2.508000,2.50,0.276428,0.500,6.680000,4.7291,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.40000,8.574000,0.400000,0.500000,1.714800
1,3.890000,0.468076,0.213892,0.046204,4.83,0.277755,0.966,4.830000,5.0000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.00000,2.554000,0.400000,0.500000,0.510800
2,3.890000,0.510000,0.200000,0.102000,0.23,0.200000,0.046,0.740000,5.0000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.00000,1.548000,0.400000,0.500000,0.309600
3,3.890000,3.901869,0.200000,1.285494,10.80,0.600000,6.480,10.800000,6.0000,0.600000,...,0.2,0.4,0.0,0.200000,0.0,4.40000,17.530000,0.400000,0.500000,3.506000
4,3.890000,0.275317,0.200000,0.065990,0.08,0.200000,0.016,0.080000,5.0000,0.200000,...,0.2,0.4,0.0,0.199514,0.0,1.89164,1.416000,0.399027,0.498784,0.283200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,27.297543,0.215336,0.200000,0.000000,0.40,0.200000,0.080,0.890000,5.0000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.00000,1.578000,0.400000,0.500000,0.315600
772,55.570000,0.470000,0.200000,0.094000,0.37,0.199514,0.074,0.583173,5.0000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.00000,1.568000,0.400000,0.500000,0.306687
773,55.570000,0.240000,0.200000,0.000000,0.00,0.200000,0.008,0.280000,5.0000,0.200000,...,0.2,0.4,0.0,0.200000,0.0,2.00000,1.456000,0.400000,0.500000,0.291200
774,55.570000,0.200000,0.198096,0.040000,0.00,0.200000,0.000,0.200000,5.0000,0.199514,...,0.2,0.4,0.0,0.200000,0.0,2.00000,1.376368,0.400000,0.500000,0.288000


In [16]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,20.082433,2.517458,0.350285,1.262276,9.410460,0.310431,6.356563,13.050038,5.051116,0.221652,...,0.205004,0.506958,0.096649,0.215347,0.045361,2.692095,15.484347,0.560093,0.498484,6.451155
std,23.787162,5.619806,0.173715,3.430712,22.071621,0.168195,30.057551,51.076782,0.266467,0.077393,...,0.033380,0.272952,0.522712,0.065042,0.287016,0.857156,38.818013,0.422243,0.009868,37.062743
min,1.496520,0.000049,0.161786,0.000000,0.000000,0.161786,0.000000,0.000049,4.044648,0.160142,...,0.161786,0.323572,0.000000,0.160142,0.000000,1.464461,0.876384,0.244077,0.404465,0.192407
25%,2.370000,0.401149,0.200000,0.000000,0.033172,0.200000,0.004358,0.608885,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.000000,1.581000,0.400000,0.500000,0.315600
50%,3.890000,0.950000,0.200000,0.147000,0.460000,0.200000,0.087000,1.370000,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,2.200000,2.200000,0.400000,0.500000,0.527021
75%,55.570000,2.547500,0.600000,1.311000,4.555000,0.400000,1.995000,7.527500,5.000000,0.200000,...,0.200000,0.400000,0.000000,0.200000,0.000000,3.200000,10.591000,0.400000,0.500000,3.242419
max,59.850000,85.000000,0.600000,51.000000,144.390000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,0.600000,2.400000,9.000000,0.600000,5.400000,5.200000,583.688000,5.800000,0.500000,961.514400


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[120   4]
 [  3  67]]
0.9639175257731959


In [22]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.1, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [  4  66]]
0.979381443298969


C:\Users\User\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
